In [ ]:
import os
import sys
import time

import GCode
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Draw a Shapes

Attempt at programmatically drawing shapes.

All units in mm. ```1``` = ```1 mm```.

In [6]:
# Draw line from (0, 0) to (10, 0)
lines = list()
line_points = np.array([[0, 0], [10, 0]])
lines.append(GCode.Line(points=line_points))

# Draw line from (0, 0) to (0, 10)
line_points = np.array([[0, 0], [0, 10]])
lines.append(GCode.Line(points=line_points))
lines

# Draw line from (0, 0) to (-10, 0)
line_points = np.array([[0, 0], [-10, 0]])
lines.append(GCode.Line(points=line_points))
lines

# Draw line from (0, 0) to (0, -10)
line_points = np.array([[0, 0], [0, -10]])
lines.append(GCode.Line(points=line_points))
lines


[<GCode>[cmds=9], <GCode>[cmds=9], <GCode>[cmds=9], <GCode>[cmds=9]]

In [20]:
class Program(GCode.GCode):
    def __init__(self, lines=list()):
        self.lines = lines

        super().__init__(*args, **kwargs)

    def generate_gcode(self):
        list(map(lambda line: line.generate_gcode(), self.lines))

    def setup(self):
        self.buffer = list()
        self.G0()
        self.G1(F=60)  # 60 mm / min = 1 mm / sec
        self.M3(
            S=1
        )  # Set laser power so that movement can be seen, but does nothing.
        self.G21()  # Metric Units
        self.G90()  # Absolute positioning.

    def teardown(self):
        self.M5()  # Power down laser.
        self.G28()  # Move home.
        # pushover("Laser is done.")

    @property
    def jog_dist(self):
        jog_dists = list()
        for idx in range(1, len(self.lines)):
            # Jog to new startpoint
            dX = self.lines[idx].x_0 - self.lines[idx - 1].x_f
            dY = self.lines[idx].y_0 - self.lines[idx - 1].y_f
            d = np.sqrt(np.power(dX, 2) + np.power(dY, 2))
            jog_dists.append(d)
        return np.cumsum(jog_dists)[-1]

    @property
    def laserin_dist(self):
        laserin_dists = [line.dist]
        return np.c

    def __repr__(self):
        return "Program<lines={}>".format(len(self.lines))


prog = Program(lines=lines)
prog.lines


[<GCode>[cmds=9], <GCode>[cmds=9], <GCode>[cmds=9], <GCode>[cmds=9]]

In [25]:
for line in prog.lines:
    print(line.dist)


[10.0]
[10.0]
[10.0]
[10.0]


In [26]:
prog.lines


[<GCode>[cmds=9], <GCode>[cmds=9], <GCode>[cmds=9], <GCode>[cmds=9]]

In [23]:
class Square(GCode.Line):
    def __init__(
        self, len_side=10, origin=np.array([0, 0]), rotation=0, *args, **kwargs
    ):
        square_points = np.array(
            [
                [origin[0], origin[1]],
                [origin[0] + len_side, origin[1]],
                [origin[0] + len_side, origin[1] + len_side],
                [origin[0], origin[1] + len_side],
                [origin[0], origin[1]],
            ]
        )

        kwargs["points"] = square_points

        super().__init__(*args, **kwargs)


s = Square()


In [9]:
s = Square(len_side=10, origin=np.array([0, 0]))
s


<GCode>[cmds=12]

In [10]:
s2 = Square(len_side=10, origin=np.array([10, 10]))
s2


<GCode>[cmds=12]

In [11]:
s.points


array([[ 0,  0],
       [10,  0],
       [10, 10],
       [ 0, 10],
       [ 0,  0]])

In [12]:
s.dist


[10.0, 10.0, 10.0, 10.0]

In [13]:
s.total_dist


40.0

In [14]:
s.origin


array([0, 0])

In [15]:
s.end


array([0, 0])

In [16]:
prog.lines[0].origin


array([0, 0])

In [17]:
prog.lines[0].end


array([10,  0])

In [19]:
self = prog


In [32]:
jog_dist = np.cumsum(jog_dists)[-1]
